In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus, get_hyphae
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children, resolve_ambiguity, relabel_nodes_after_amb
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from amftrack.pipeline.functions.node_id import reconnect_degree_2
from amftrack.pipeline.functions.extract_graph import (
    generate_skeleton,
    from_nx_to_tab,
    prune_graph
)

from amftrack.pipeline.functions.hyphae_id_surf import (
    clean_and_relabel,
    get_mother,
    save_hyphaes,
    resolve_ambiguity_two_ends,
    clean_obvious_fake_tips,
    width_based_cleaning
)
from amftrack.pipeline.functions.experiment_class_surf import clean_exp_with_hyphaes
from scipy.ndimage import gaussian_filter
from scipy import ndimage
from scipy.optimize import curve_fit
from shapely.geometry import Polygon, shape
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util import get_dates_datetime, get_dirname, get_plate_number, get_postion_number,get_begin_index
import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment, Edge, Node, Hyphae, plot_raw_plus, get_hyphae
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project
from amftrack.notebooks.analysis.util import *
from scipy import stats
from scipy.ndimage.filters import uniform_filter1d
from statsmodels.stats import weightstats as stests
from amftrack.pipeline.functions.hyphae_id_surf import get_pixel_growth_and_new_children, resolve_ambiguity, relabel_nodes_after_amb
from collections import Counter
from IPython.display import clear_output
from amftrack.notebooks.analysis.data_info import *
from amftrack.pipeline.functions.node_id import reconnect_degree_2
from amftrack.pipeline.functions.extract_graph import (
    generate_skeleton,
    from_nx_to_tab,
    prune_graph
)

from amftrack.pipeline.functions.hyphae_id_surf import (
    clean_and_relabel,
    get_mother,
    save_hyphaes,
    resolve_ambiguity_two_ends,
    clean_obvious_fake_tips,
    width_based_cleaning
)
from amftrack.pipeline.functions.experiment_class_surf import clean_exp_with_hyphaes
from scipy.ndimage import gaussian_filter
from scipy import ndimage
from scipy.optimize import curve_fit
from shapely.geometry import Polygon, shape
directory = directory_project
plate_number =94
plate = get_postion_number(plate_number)
begin = 269
end = 329
dates_datetime = get_dates_datetime(directory,plate)
dates = dates_datetime[begin:end+1]
exp = Experiment(plate,directory)
exp.load(dates) #for method 2
lengths = [get_length(exp,t) for t in range(exp.ts)]
times = [get_time(exp,0,t) for t in range(exp.ts)]
lengths = np.array(lengths)
times = np.array(times)
areas = []   
for t in range(exp.ts):
    nodes = np.array([node.pos(t) for node in exp.nodes if node.is_in(t)])
    hull=scipy.spatial.ConvexHull(nodes)
    poly = Polygon([nodes[vertice] for vertice in hull.vertices])
    areas.append(poly.area* 1.725**2/(1000**2))
speeds = (lengths[1:]-lengths[:-1])/(times[1:]-times[:-1])
end = 16
begin = 0
lengths_select = lengths[begin:end]
times_select = times[begin:end]
areas_select = areas[begin:end]

coefs = np.polyfit(times_select,areas_select ,2)
p = np.poly1d(coefs)
def func(t,a,t0):
    return(a*(t-t0)**2)
popt0, pcov = curve_fit(func, times_select,areas_select ,bounds = ([0,-np.inf],2*[np.inf]),p0=[1,1])
def func2(t,alpha,C,d):
    return(C*(np.exp(alpha*(t)))+d)
popt1, pcov = curve_fit(func2, times_select, lengths_select/1000,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])


In [2]:
end = 30
begin = 0
lengths_select = lengths[begin:end]
times_select = times[begin:end]
areas_select = areas[begin:end]

coefs = np.polyfit(times_select,areas_select ,2)
p = np.poly1d(coefs)
def func(t,a,t0):
    return(a*(t-t0)**2)
popt0, pcov = curve_fit(func, times_select,areas_select ,bounds = ([0,-np.inf],2*[np.inf]),p0=[1,1])
def func2(t,alpha,C,d):
    return(C*(np.exp(alpha*(t)))+d)
popt1, pcov = curve_fit(func2, times_select, lengths_select/1000,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])


In [13]:
plt.close('all')
plt.rcParams.update({
    "font.family": "verdana",
'font.weight' : 'normal',
'font.size': 12})

fig=plt.figure(figsize=(9,8))
ax = fig.add_subplot(111)
ax.scatter(times_select,areas_select ,label = 'data areas ($mm^2$)')
ax.plot(times_select,p(np.array(times_select)), label= f'quadratic fit a={"{:.2f}".format(coefs[0])} mm^2.h-2')
# ax.plot(times,func(np.array(times),*popt0),label = f'quadratic fit a={"{:.2f}".format(popt0[0])} mm^2.h-2')
ax.set_xlabel('time (h)',fontdict={'size' : 20})
ax.set_ylabel('network area $(mm^2)$',fontdict={'size' : 20})
plt.legend()
ax2 = ax.twinx()
ax2.scatter(times_select,lengths_select/1000,color='red', label = 'data length ($mm$)')
ax2.plot(times_select,func2(np.array(times_select),*popt1),label = f'exponential fit : alpha= {"{:.2f}".format(popt1[0]*24)}.day-1',color='red')

ax2.set_ylabel('network length $(mm)$',fontdict={'size' : 20})

plt.legend(loc = 'upper right',ncol=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …